# Import

In [ ]:
import pandas as pd

# Load

In [ ]:
data = pd.read_csv('3_clean.csv')
data

# Analyze

In [ ]:
# Visualize the completeness of the different cases.
completeness = data.count(axis=1) / len(data.columns) * 100
completeness = completeness.apply(lambda x: round(x / 10) * 10)
completeness = completeness.value_counts().sort_index()
completeness.plot.bar()

# Exclusion Criteria

In [ ]:
# Urgency
data = data[data['meta_urgency'] >= 4]
data

In [ ]:
# 30 Day Mortality
data = data[data['target_30_day_mortality'].notna()]
data

In [ ]:
# 90 Day Mortality
data = data[data['target_90_day_mortality'].notna()]
data

In [ ]:
# 75% Completeness
data = data.dropna(thresh=len(data.columns) * 0.75)
data

# Cut

In [ ]:
data = data.loc[:, ~data.columns.str.startswith('meta_')]
data = data.loc[:, ~data.columns.str.startswith('target_') | data.columns.str.startswith('target_30_day_mortality') | data.columns.str.startswith('target_90_day_mortality')]
data

# Save

In [ ]:
data.to_csv('6_filtered.csv', index = False)
data

In [ ]:
description: pd.DataFrame = pd.DataFrame()

i = 0
for column in data.columns:
    data_total = data[column]
    data_training = data[data['identifier_cohort'] == 0][column]
    data_validation = data[data['identifier_cohort'] == 1][column]

    description = description.append({
        'category': '...',
        'id': i,
        'name': column,
        'dimension': '...',
        'completeness': round(data_total.count() / len(data_total) * 100, 2),
        'completeness (0)': round(data_training.count() / len(data_training) * 100, 2),
        'completeness (1)': round(data_validation.count() / len(data_validation) * 100, 2),
        'min': data_total.min(),
        'min (0)': data_training.min(),
        'min (1)': data_validation.min(),
        'max': data_total.max(),
        'max (0)': data_training.max(),
        'max (1)': data_validation.max(),
        'lower_limit': '...',
        'upper_limit': '...',
    }, ignore_index=True)

    i += 1

description.to_csv('7_description.csv', index=False)
description